In [7]:
from environment import Environment, VMConfig
from smolagents import CodeAgent, HfApiModel
import docker
import logging
import os
from pathlib import Path

In [8]:
model_id = "meta-llama/Llama-3.3-70B-Instruct"
agent = CodeAgent(name="TestAgent", tools=[], model=HfApiModel(model_id=model_id), add_base_tools=True)


## Setting up the Environment

This notebook demonstrates how to set up a virtualized environment for testing using temporary directories.


In [ ]:
# Set up the config with the temporary directory
ROOT = Path(os.getcwd()).parent
print(f"Root directory: {ROOT}")

# Check for any existing containers that might cause port conflicts
docker_client = docker.from_env()
containers = docker_client.containers.list(all=True)
print(f"Existing containers: {containers}")

# Create a configuration with a higher starting port range to avoid conflicts
ubuntu_config = VMConfig(
    vm_os="ubuntu",
    vm_ram_size="4G",
    vm_cpu_cores="4",
    vm_disk_size="20G",
    default_port_ranges={"vnc": 9006, "server": 9765},  # Use higher port numbers
)

print("Configuration details:")
print(f"Project root: {ubuntu_config.project_root}")
print(f"Volumes directory: {ubuntu_config.volumes_directory}")
print(f"VM environments: {ubuntu_config.vm_environments}")
print(f"Starting port ranges: {ubuntu_config.default_port_ranges}")

# Create environment with the config
env = Environment(config=ubuntu_config)
# env.start_vm()


Root directory: /home/smwhering/IS-master-thesis
Existing containers: []
Configuration details:
Project root: /home/smwhering/IS-master-thesis/src
Volumes directory: /home/smwhering/IS-master-thesis/src/docker/volumes
VM environments: /home/smwhering/IS-master-thesis/src/docker/volumes/environments
Starting port ranges: {'vnc': 9006, 'server': 9765}


Copying VM files: 0files [00:00, ?files/s]
Error waiting for WebSocket server to be ready: WebSocket server failed to become ready within 300s timeout period
Failed to start VM: WebSocket server failed to become ready within 300s timeout period


TimeoutError: WebSocket server failed to become ready within 300s timeout period